In [2]:
import json
import os
import numpy as np
import re
import nltk
import stanza

In [89]:
stanza_tokenizer = stanza.Pipeline("en")

2024-06-12 15:33:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-12 15:33:40 INFO: Downloaded file to /home/nchibbar/stanza_resources/resources.json
2024-06-12 15:33:41 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-06-12 15:33:41 INFO: Using device: cuda
2024-06-12 15:33:41 INFO: Loading: tokenize
2024-06-12 15:33:42 INFO: Loading: mwt
2024-06-12 15:33:42 INFO: Loading: pos
2024-06-12 15:33:43 INFO: Loading: lemma
2024-06-12 15:33:43 INFO: Loading: constituency
2024-06-12 15:33:43 INFO: Loading: depparse
2024-06-12 15:33:43 INFO: Loading: sentiment
2024-06-12 15:33:44 INFO: Loading: ne

In [19]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [2]:
sample_text = """Hey there! 😊 Have you checked out the new website? Visit us at https://www.example.com or email us at info@example.com for more details. #ExcitingNews 📰

The meeting is scheduled for 12:30 PM on 15th August, 2023. Don't be late! 🚀 We’ll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%. 📊

Here are the main points:

Increase marketing budget by 25% 🚀
Launch the new product line in Q4 🎉
Expand into the APAC region 🌏

To-do List:

 Finalize the budget proposal
 Review the Q3 report
 Prepare for the product launch event

Alice said, “We need to push our limits!” while Bob replied, “Let’s ensure quality over quantity.” This debate has been ongoing for weeks now. Some say, "Quality is key," but others argue, "We need quantity to drive growth."

Contact our support team at support@example.com or call 1-800-555-0199. For international calls, dial +1-800-555-0199.

Here's a snippet of code we need to review:

def hello_world():
    print("Hello, World!")

Make sure to check it and ensure it’s compliant with PEP 8 standards. Also, we have some legacy code:

<!DOCTYPE html>
<html>
<body>
    <h1>My First Heading</h1>
    <p>My first paragraph.</p>
</body>
</html>

Random thoughts:

I can't believe it's almost the end of the year!
What are everyone's New Year's resolutions? 🎆
The weather has been crazy lately, right? 🌦️
Check out this great article: https://www.example.com/blog/awesome-article. It's packed with useful info and insights.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Sed cursus ante dapibus diam. Sed nisi. Nulla quis sem at nibh elementum imperdiet. Duis sagittis ipsum. Praesent mauris. Fusce nec tellus sed augue semper porta. Mauris massa.

Nullam quis risus eget urna mollis ornare vel eu leo.
Integer posuere erat a ante venenatis dapibus posuere velit aliquet.
Special characters to consider: !@#$%^&*()_+-=[]{}|;':",.<>/?`~

Do you remember the last project's details? If not, here's a quick refresher: the budget was $500,000, and the timeline was 6 months. We faced several challenges, including supply chain issues and unexpected delays. Yet, we managed to deliver on time. Kudos to the team! 🎉

By the way, here’s a fun fact: the longest recorded flight of a chicken is 13 seconds! 🐔 How cool is that?

Our next team-building activity will be a hiking trip. Bring your gear and be ready for an adventure. 🌄

Hiking Essentials:
Sturdy boots 🥾
Water bottle 🚰
Snacks 🍫
First-aid kit 🩹
The goal is to reach the summit by noon. Let's make it happen, team! 💪
"""

In [85]:
def preprocess_text(text, stop_words=None):

	# Convert non-ASCII quotes to ASCII quotes
	text = re.sub(r"‘|’", "'", text)
	text = re.sub(r"“|”", '"', text)

	# Remove non-ASCII characters
	text = re.sub(r"[^\x00-\x7f]+", "", text)

	# Remove emails
	text = re.sub(r"[^\s]+@[^\s]+\.com", "", text)

	# Remove hyperlinks
	text = re.sub(r"[^\s]*://[^\s]*", "", text)

	# Remove hashtags
	text = re.sub(r"#[^\s]+", "", text)

	# Remove HTML tags
	text = re.sub(r"<[^\n>]+>", "", text)

	# Remove numbers
	# text = re.sub(r"[+?\d+-?]+", "", text)

	# Remove stop words
	if stop_words:
		text = re.sub(r"|".join([
			rf"\s{word}[\s.]" for word in stop_words
		]), " ", text)

	# Concatenating multiple spaces and tabs
	text = re.sub(r"[ \t]{2,}", " ", text)

	# Removing spaces and tabs before newline
	text = re.sub(r"[ \t]\n", "\n", text)

	# Concatenating multiple newlines
	text = re.sub(r"\n{3,}", "\n\n", text)

	return text

In [86]:
preprocessed_text = preprocess_text(sample_text)
# preprocessed_text = preprocess_text(sample_text, stop_words=stopwords)
print(preprocessed_text)

Hey there! Have you checked out the new website? Visit us at or email us at for more details.

The meeting is scheduled for 12:30 PM on 15th August, 2023. Don't be late! We'll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%.

Here are the main points:

Increase marketing budget by 25%
Launch the new product line in Q4
Expand into the APAC region

To-do List:

 Finalize the budget proposal
 Review the Q3 report
 Prepare for the product launch event

Alice said, "We need to push our limits!" while Bob replied, "Let's ensure quality over quantity." This debate has been ongoing for weeks now. Some say, "Quality is key," but others argue, "We need quantity to drive growth."

Contact our support team at or call 1-800-555-0199. For international calls, dial +1-800-555-0199.

Here's a snippet of code we need to review:

def hello_world():
 print("Hello, World!")

Make sure to check it and ensure it's compliant with PEP 8 standards. Also, we have some le

In [87]:
nltk.sent_tokenize(preprocessed_text)

['Hey there!',
 'Have you checked out the new website?',
 'Visit us at or email us at for more details.',
 'The meeting is scheduled for 12:30 PM on 15th August, 2023.',
 "Don't be late!",
 "We'll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%.",
 'Here are the main points:\n\nIncrease marketing budget by 25%\nLaunch the new product line in Q4\nExpand into the APAC region\n\nTo-do List:\n\n Finalize the budget proposal\n Review the Q3 report\n Prepare for the product launch event\n\nAlice said, "We need to push our limits!"',
 'while Bob replied, "Let\'s ensure quality over quantity."',
 'This debate has been ongoing for weeks now.',
 'Some say, "Quality is key," but others argue, "We need quantity to drive growth."',
 'Contact our support team at or call 1-800-555-0199.',
 'For international calls, dial +1-800-555-0199.',
 'Here\'s a snippet of code we need to review:\n\ndef hello_world():\n print("Hello, World!")',
 "Make sure to check it and

In [90]:
parsed_text = stanza_tokenizer(preprocessed_text)

In [100]:
[sentence.text for sentence in parsed_text.sentences]

['Hey there!',
 'Have you checked out the new website?',
 'Visit us at or email us at for more details.',
 'The meeting is scheduled for 12:30 PM on 15th August, 2023.',
 "Don't be late!",
 "We'll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%.",
 'Here are the main points:',
 'Increase marketing budget by 25%\nLaunch the new product line in Q4\nExpand into the APAC region',
 'To-do List:',
 'Finalize the budget proposal\n Review the Q3 report\n Prepare for the product launch event',
 'Alice said, "We need to push our limits!" while Bob replied, "Let\'s ensure quality over quantity."',
 'This debate has been ongoing for weeks now.',
 'Some say, "Quality is key," but others argue, "We need quantity to drive growth."',
 'Contact our support team at or call 1-800-555-0199.',
 'For international calls, dial +1-800-555-0199.',
 "Here's a snippet of code we need to review:",
 'def hello_world():\n print("Hello, World!")',
 "Make sure to check it and 

## gov-report

### crs

In [27]:
data_dir = "/home/nchibbar/Data/GovReport"
all_files = os.listdir(data_dir)
len(all_files)

7238

In [4]:
file = all_files[1]
file = f"{data_dir}/{file}"

In [5]:
with open(file) as fp:
	data = json.load(fp)

data.keys()

dict_keys(['id', 'title', 'released_date', 'summary', 'reports'])

In [16]:
data["reports"]["subsections"][0].keys()

dict_keys(['section_title', 'paragraphs', 'subsections'])

In [7]:
count_words = lambda text_list: len(" ".join(text_list).split())

In [14]:
def parse_subsections(subsection):
	words = subsection["paragraphs"]
	for sub in subsection["subsections"]:
		words = f"{words} {parse_subsections(sub)}"
	return words

In [15]:
parse_subsections(data["reports"])

12474

In [24]:
avg_words, avg_summary_words = 0, 0
num_files = len(all_files)

for file in all_files:
	with open(f"{data_dir}/{file}") as fp:
		data = json.load(fp)
	avg_words += parse_subsections(data["reports"])	
	avg_summary_words += count_words(data["summary"])

avg_words / num_files, avg_summary_words / num_files

(7564.149212489638, 451.45537441282124)

### gao

In [28]:
data_dir = "/Users/naman/Workspace/Data/UCCS-REU/gov-report/gao"
all_files = os.listdir(data_dir)
len(all_files)

12228

In [59]:
file = all_files[0]

with open(f"{data_dir}/{file}") as fp:
	data = json.load(fp)

data.keys()

dict_keys(['id', 'url', 'title', 'published_date', 'released_date', 'highlight', 'report', 'fastfact'])

In [55]:
data["report"][0].keys()

dict_keys(['section_title', 'paragraphs', 'subsections'])

In [60]:
words = 0
for report in data["report"]:
	words += parse_subsections(report)
words

9527

In [56]:
parse_subsections(data["report"][0])

8781

In [57]:
words = 0
for file in all_files:
	with open(f"{data_dir}/{file}") as fp:
		data = json.load(fp)
	for report in data["report"]:
		words += parse_subsections(report)
words / len(all_files)

8521.906771344455

In [74]:
with open("ideas.txt") as fp:
	file_iter = iter(fp.readlines())

['## My ideas\n', '\n', 'Generating multiple words at a time in AR transformers to improve efficiency\n', '\n', 'Sentence level attention mechanism for summarizing long inputs\n', '\n', 'Compression of sentences (maybe by using discourse theory) and extraction for summarization\n', '\n', 'Summarize iteratively evaluating and improving each summary\n', '\n', 'Extraction based on topic words\n', '\n', 'QA with long input contexts by uniformly sampling sentences or phrases\n', '\n', 'Employ speed reading techniques\n', 'Skimming: Quickly scan over the whole text skipping some parts\n', 'Chunking: Break text into chunks and go over each one individually\n', 'Pacing: Read many words at once\n', 'Eliminating: Ignore redundant parts of text while reading\n', '\n', 'Summarizing books based on chapter names or title\n', '\n', 'Use 1D convolution to reduce number of word embeddings\n', '\n', 'Truncating from the middle instead of extremeties\n', '\n', '\n', "## Papers' ideas\n", '\n', 'Long and 

In [5]:
re.sub(r"\sthe([\s.])", r"\1", "hey the. watch the this!")

'hey. watch this!'